In [1]:
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
import os
os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = str(pow(2,40))

In [2]:
import aPack_model
from configs import sentinel_2

/opt/conda/lib/python3.10/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.12 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations


/data/00.Data/AO/0-Pusan-2023-AO/C1_20230116015105_10110_00006119_L1G_PS
time :  2023 1 16 10 51
202301161000_dynamic.csv
/data/00.Data/AIS-CSV-NTO-AO/2차-11개/202301161000_dynamic.csv
#------------------ : csv exists


In [3]:
'''
1. args : 이미지 경로, ais csv 경로, 탐색 옵션 등 매칭 관련 정보들 조절 
2. model_config_file : mmrotate 에서 불러들일 모델에 대한 내용들
3. check_point_file : ckpt 파일경로 
'''

args = sentinel_2
model_config_file = '/root/Ship_Detection_EO/configs/roi_trans.py'
check_point_file = '/root/Ship_Detection_EO/model_ckpts/epoch_9.pth'

In [4]:
args.model_test

True

In [5]:
args.img_path

'/data/00.Data/AO/0-Pusan-2023-AO/C1_20230116015105_10110_00006119_L1G_PS/0-Pusan-2023-AO_3_C1_20230116015105_10110_00006119_L1G.png'

In [6]:
img = cv2.imread(args.img_path)
print(img.shape)

(28880, 29600, 3)


In [7]:
# --------------------------------------------------------------------------------
# model inference
# --------------------------------------------------------------------------------
if args.model_test: 
    result = aPack_model.infer_model(
        input_file_path=args.img_path,
        checkpoint_file=check_point_file,
        config_file=model_config_file,
        thres=args.infer_threshold , 
        gpu_id = 0 )

/opt/conda/lib/python3.10/site-packages/mmdet/models/dense_heads/anchor_head.py:116: UserWarning: DeprecationWarning: `num_anchors` is deprecated, for consistency or also use `num_base_priors` instead
  warnings.warn('DeprecationWarning: `num_anchors` is deprecated, '


load checkpoint from local path: /root/Ship_Detection_EO/model_ckpts/epoch_9.pth
#------------------- init detection
# ---- batch_size ---------------------------- # :  32
# ---- image sizes ---------------------------- # :  [1024]
# ---- steps ---------------------------- # :  [896]


/opt/conda/lib/python3.10/site-packages/mmdet/models/dense_heads/anchor_head.py:123: UserWarning: DeprecationWarning: anchor_generator is deprecated, please use "prior_generator" instead
  warnings.warn('DeprecationWarning: anchor_generator is deprecated, '


In [8]:
# --------------------------------------------------------------------------------
# Save result
# --------------------------------------------------------------------------------

save_root = "/root/Ship_Detection_EO/outputs"
file_name = args.img_path.split("/")[-1].split(".")[0]+".npy"    

if args.model_test == False:
    result = np.load(os.path.join(save_root,file_name))
else:
    print("Result Saved")
    np.save(os.path.join(save_root,file_name), result)

Result Saved


In [9]:
result

[array([[ 2.1798e+04,  2.1492e+03,  4.8351e+01,  4.5629e+01,  5.0373e-02,  9.9669e-01],
        [ 2.1798e+04,  2.0473e+03,  4.8364e+01,  4.5905e+01,  7.8270e-02,  9.9631e-01],
        [ 2.2102e+04,  1.5303e+04,  5.2105e+01,  5.0903e+01, -8.1607e-03,  9.9574e-01],
        ...,
        [ 1.0684e+04,  2.1472e+04,  2.8700e+01,  2.7924e+01, -1.6678e-01,  5.0966e-02],
        [ 1.4311e+04,  2.4156e+04,  2.4162e+01,  2.1587e+01,  1.4466e+00,  5.0528e-02],
        [ 6.3621e+03,  9.0215e+03,  2.5846e+01,  1.9117e+01,  1.4125e+00,  5.0343e-02]], dtype=float32)]

In [10]:
import torch
torch.cuda.empty_cache()

In [11]:
# --------------------------------------------------------------------------------
# Bbox Show and Save 
# --------------------------------------------------------------------------------

img_path = args.img_path
img_ = Image.open(img_path)

bboxes_ = [] 
cnt = 0

print("#--- inference threshold : ", args.infer_threshold)
for r in result[0]:
    if r[-1] > args.infer_threshold:
        cnt+=1
        bboxes_.append(r)

plt.figure(figsize=(18,18))
plt.imshow(img_)
ax = plt.gca()

box_cnt = 0
for box_ in bboxes_:
    box_cnt+=1
    print(f"#-------------- {box_cnt / len(bboxes_):.4f}")
    x = box_[0]
    y = box_[1]
    width = box_[2]
    height = box_[3]
    angle_ = box_[4]

    # model 의 객체좌표가 center 중심인데, patches.Rectangle 은 좌하단을 기준점으로 보기 때문에. x,y 를 아래와 같이 shift 
    # roatate 할 때, 중심좌표 center 로 잡고, angle radian 변환해서 넣어주기 
    path_obj = patches.Rectangle((x-width*0.5,y-height*0.5), width, height, linewidth=0.1, edgecolor="red", fill=False,
                                rotation_point="center",
                                angle=angle_*180/np.pi)
    ax.add_patch(path_obj)

plt.savefig("/root/Ship_Detection_EO/results/testv1",dpi =800 )
plt.show()

DecompressionBombError: Image size (854848000 pixels) exceeds limit of 178956970 pixels, could be decompression bomb DOS attack.